In [14]:
# Part 1
#For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

#Start by creating a new Notebook for this assignment.
#Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

# I pulled up the web page, highlighted the table, cut & pasted to Excel and saved to a csv file, "Toronto.csv".
# I then imported the file into IBM Cloud Storage and used "Insert to code" function to create the following code:

# Start of "Insert to code segment"
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e4c63f1d09c549968caf743ebebfff9a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='d73P9G0wt07N6duy6tsqsYmwESOQl5apjZcKB3bEct-2',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_e4c63f1d09c549968caf743ebebfff9a.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-tbbplt7t0dhxdc',Key='Toronto.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
# End of "Insert to code segment"
    
df = pd.read_csv(body)
print("Original Shape", df.shape)

#Set width so output isn't split
pd.set_option('display.width', 150)
df.head()


Original Shape (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
is_Notassigned =  df['Neighbourhood']=='Not assigned'
dfNa = df[is_Notassigned]
dfNa_count = dfNa.count
if dfNa.empty:
    print("0 remaining rows have Neighborhood = Not assigned\n")
else:
    print(dfNa_count, "remaining rows have Nieghborhood = Not assigned\n")
    
#There are no rows left where Neighbourhood is "Not assigned"

#More than one neighborhood can exist in one postal code area. 
#Combine rows with the same postal code area into one row with the neighborhoods separated with a comma.
#The Wikipedia page presented no such rows, looks like they had already been combined!
#Just for yucks, I will check for duplicate rows based on the postal area code.
dfDup = df[df.duplicated(['Postal Code'])]
if dfDup.empty:
    print("There are 0 duplicate rows in the dataframe\n")
else:
    print("There are", dfDup.count, "duplicate rows in the dataframe\n")
print("Filtered Shape", df.shape, "\n")

df = df.reset_index()
del df['index']
df.head()

0 remaining rows have Neighborhood = Not assigned

There are 0 duplicate rows in the dataframe

Filtered Shape (103, 3) 



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
# Part 2
#Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, 
#in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

import pandas as pd
dfCoord = pd.read_csv('http://cocl.us/Geospatial_data')
print(dfCoord.head())
neighborhoods = pd.merge(df, dfCoord, on='Postal Code')
print(neighborhoods.head())
print(neighborhoods.shape)

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
  Postal Code           Borough                                Neighbourhood   Latitude  Longitude
0         M3A        North York                                    Parkwoods  43.753259 -79.329656
1         M4A        North York                             Victoria Village  43.725882 -79.315572
2         M5A  Downtown Toronto                    Regent Park, Harbourfront  43.654260 -79.360636
3         M6A        North York             Lawrence Manor, Lawrence Heights  43.718518 -79.464763
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government  43.662301 -79.389494
(103, 5)


In [17]:
#Part3
#Once the data is in a structured format, you can replicate the analysis 
#that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

#--------------------------------------------------------------------------------
# NOTE: I had to comment the conda installs below and create a custom environment
#       in IBM Watson to get this section to work:
# CUSTOM ENVIRONMENT:
#    channels: 
#    - defaults 
#    - conda-forge 
#    dependencies: 
#    - geopandas=0.3.0 
#    - geoplot=0.2.3 
#    - pysal=1.14.4 
#    - folium=0.7.0 
#    - python=3.6.8 
#--------------------------------------------------------------------------------

#Prepare for analysis
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

###!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

###!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [26]:
#Verify the data set
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

#Verify the location for downtown Toronto
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")

location = geolocator.geocode(address)
print(location)

latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
#    label = folium.Popup(label, parse_html=True)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

The dataframe has 10 boroughs and 103 neighborhoods.
Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada
The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [30]:
# Now I explore downtown Toronto further with the FourSquare api
CLIENT_ID = 'JSCXMJRKCCMGJ5A1QZLG20UIUXII5UYPSY0KV4LSORXHU5AK' # your Foursquare ID
CLIENT_SECRET = 'OLMSOKTFUOKIEXKC45YTCSCB5ICEEYEOUMEIVZN25QGM1V3Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2b18d1fe1f6657f6a3daa3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 92,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
venues = results['response']['groups'][0]['items'] 

from pandas.io.json import json_normalize

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.head(), "\n")
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]), "\n")


                                name        categories        lat        lng
0                   Downtown Toronto      Neighborhood  43.653232 -79.385296
1             Nathan Phillips Square             Plaza  43.652270 -79.383516
2                            Japango  Sushi Restaurant  43.655268 -79.385165
3                          Poke Guys        Poke Place  43.654895 -79.385052
4  Eggspectation Bell Trinity Square    Breakfast Spot  43.653144 -79.381980 

92 venues were returned by Foursquare. 



In [48]:
# Now cluster the Toronto neighborhoods

# GetNearbyVenues function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:

toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [108]:
print(toronto_venues.shape)
print(toronto_venues.head())
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.set_index('Neighborhood')

neighborhoods_merged = neighborhoods
neighborhoods_merged.set_index('Neighbourhood')

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
neighborhoods_merged = neighborhoods_merged[neighborhoods_merged['Cluster Labels'].notna()]
neighborhoods_merged["Cluster Labels"] = neighborhoods_merged["Cluster Labels"].astype(int)

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Neighbourhood'], neighborhoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

(2151, 7)
       Neighborhood  Neighborhood Latitude  Neighborhood Longitude                   Venue  Venue Latitude  Venue Longitude         Venue Category
0         Parkwoods              43.753259              -79.329656         Brookbanks Park       43.751976       -79.332140                   Park
1         Parkwoods              43.753259              -79.329656           Variety Store       43.751974       -79.333114      Food & Drink Shop
2  Victoria Village              43.725882              -79.315572  Victoria Village Arena       43.723481       -79.315635           Hockey Arena
3  Victoria Village              43.725882              -79.315572               Portugril       43.725819       -79.312785  Portuguese Restaurant
4  Victoria Village              43.725882              -79.315572             Tim Hortons       43.725517       -79.313103            Coffee Shop
There are 275 unique categories.


In [109]:
# Examine the clusters
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 0, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,French Restaurant,Coffee Shop,Intersection,Pizza Place,Portuguese Restaurant,Hockey Arena,Diner,Dance Studio,Deli / Bodega,Department Store
10,North York,0,Park,Pizza Place,Pub,Japanese Restaurant,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
50,North York,0,Gym,Furniture / Home Store,Pizza Place,Home Service,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
70,Etobicoke,0,Pizza Place,Discount Store,Middle Eastern Restaurant,Coffee Shop,Intersection,Sandwich Place,Chinese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant
77,Etobicoke,0,Park,Pizza Place,Sandwich Place,Bus Line,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
93,Etobicoke,0,Pizza Place,Pharmacy,Pub,Pool,Sandwich Place,Coffee Shop,Skating Rink,Gym,Distribution Center,Discount Store


In [ ]:
#....and so on